In [ ]:
!pip install opencv-python matplotlib seaborn panda numpy sklearn

In [7]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras.layers import LSTM, Dense, Dropout
from keras.models import Sequential
from sklearn.model_selection import train_test_split

In [8]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [9]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [13]:
nodding_df = pd.read_csv("../mediapipe/csv_data/NODDING.csv")
handwave_df = pd.read_csv("../mediapipe/csv_data/HANDWAVE.csv")

In [35]:
no_of_timestep = 10
X = []
y = []

# Define labels
# 0 - nodding
# 1 - handwave
nodding_data = nodding_df.iloc[:,:].values
for i in range(no_of_timestep, len(nodding_data)):
    X.append(nodding_data[i-no_of_timestep:i,:])
    y.append(0)

handwave_data = handwave_df.iloc[:,:].values
for i in range(no_of_timestep, len(handwave_data)):
    X.append(handwave_data[i-no_of_timestep:i,:])
    y.append(1)

X, y = np.array(X), np.array(y)
X.shape

(1180, 10, 132)

In [38]:
# Architect model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation="sigmoid"))
model.compile(optimizer="adam", metrics = ['accuracy'], loss = "binary_crossentropy")

In [42]:
history = model.fit(X_train, y_train, epochs=16, batch_size=32,validation_data=(X_test, y_test))

Epoch 1/16
30/30 [==============================] - 1s 27ms/step - loss: 1.3969e-04 - accuracy: 1.0000 - val_loss: 9.0034e-05 - val_accuracy: 1.0000
Epoch 2/16
30/30 [==============================] - 1s 24ms/step - loss: 1.2318e-04 - accuracy: 1.0000 - val_loss: 8.2456e-05 - val_accuracy: 1.0000
Epoch 3/16
30/30 [==============================] - 1s 23ms/step - loss: 1.1914e-04 - accuracy: 1.0000 - val_loss: 7.5860e-05 - val_accuracy: 1.0000
Epoch 4/16
30/30 [==============================] - 1s 23ms/step - loss: 1.0640e-04 - accuracy: 1.0000 - val_loss: 7.0224e-05 - val_accuracy: 1.0000
Epoch 5/16
30/30 [==============================] - 1s 22ms/step - loss: 1.0385e-04 - accuracy: 1.0000 - val_loss: 6.5287e-05 - val_accuracy: 1.0000
Epoch 6/16
30/30 [==============================] - 1s 22ms/step - loss: 8.9522e-05 - accuracy: 1.0000 - val_loss: 6.1147e-05 - val_accuracy: 1.0000
Epoch 7/16
30/30 [==============================] - 1s 22ms/step - loss: 9.1472e-05 - accuracy: 1.0000 - v

In [ ]:
# loss
fig = plt.figure()
plt.plot(history.history['loss'], color='teal', label='loss')
plt.plot(history.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
# Accuracy
fig = plt.figure()
plt.plot(history.history['accuracy'], color='teal', label='accuracy')
plt.plot(history.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [46]:
model.save('./models/nodwave.h5')